In [1]:
from sponge import Sponge
from sponge import ForceField
from sponge.optimizer import SteepestDescent
from sponge.callback import WriteH5MD, RunInfo
from sponge import set_global_units
from sponge import Protein
from sponge import UpdaterMD
from sponge.function import VelocityGenerator
from mindspore import context

In [2]:
context.set_context(mode=context.PYNATIVE_MODE, device_target="GPU")

## 单位设置

这里我们提到的全局单位设置，主要是指输入输出的长度单位和能量单位，在MindSPONGE的计算过程中会自动根据默认单位进行计算，再将返回的结果转换成用户定义的单位。常用的长度单位是nm和A，一般在PDB文件里面是用A为单位。能量单位比较常用的是kJ/mol和kcal/mol。

In [3]:
set_global_units('nm', 'kj/mol')

## 蛋白质案例

我们提供了一些简单的蛋白质案例以供测试，一般用pdb(Protein Data Bank)格式来存储蛋白质文件。下述案例就是一个不含H原子的pdb文件（该构象来自于MEGAProtein的预测结果）。

![](../../docs/case2-1.png)

在使用MindSPONGE构建Protein对象时，会自动为其在相对合理的位置补充H原子（如下图所示）。而这一过程之后，最好对其进行一次能量最小化的操作，以确保得到的构象中不会存在太多的冲突，比如两个原子距离太近等。

![](../../docs/case2-3.png)

如果使用常用的蛋白质可视化算法NewCartoon来展示的话，可以更清晰的看到这个构象中所存在的$\alpha$螺旋和$\beta$折叠等区域：

![](../../docs/case2-2.png)

In [4]:
pdb_name = 'case2.pdb'
system = Protein(pdb=pdb_name, rebuild_hydrogen=True)

[MindSPONGE] Adding 2529 hydrogen atoms for the protein molecule in 1.113 seconds.


In [5]:
energy = ForceField(system, 'AMBER.FF14SB')

In [6]:
min_opt = SteepestDescent(system.trainable_params(), 1e-7)

In [7]:
md = Sponge(system, energy, min_opt)

[MindSPONGE] Using fully connected neighbour list (not updated).


In [8]:
run_info = RunInfo(10)
md.run(500, callbacks=[run_info])

[MindSPONGE] Started simulation at 2023-06-30 16:48:59
[MindSPONGE] Step: 0, E_pot: 141277.78
[MindSPONGE] Step: 10, E_pot: 96861.06
[MindSPONGE] Step: 20, E_pot: 75968.81
[MindSPONGE] Step: 30, E_pot: 62559.87
[MindSPONGE] Step: 40, E_pot: 53021.83
[MindSPONGE] Step: 50, E_pot: 45810.742
[MindSPONGE] Step: 60, E_pot: 40129.08
[MindSPONGE] Step: 70, E_pot: 35518.984
[MindSPONGE] Step: 80, E_pot: 31696.498
[MindSPONGE] Step: 90, E_pot: 28473.77
[MindSPONGE] Step: 100, E_pot: 25719.998
[MindSPONGE] Step: 110, E_pot: 23340.135
[MindSPONGE] Step: 120, E_pot: 21262.908
[MindSPONGE] Step: 130, E_pot: 19433.643
[MindSPONGE] Step: 140, E_pot: 17809.637
[MindSPONGE] Step: 150, E_pot: 16357.127
[MindSPONGE] Step: 160, E_pot: 15049.16
[MindSPONGE] Step: 170, E_pot: 13864.028
[MindSPONGE] Step: 180, E_pot: 12784.107
[MindSPONGE] Step: 190, E_pot: 11794.958
[MindSPONGE] Step: 200, E_pot: 10884.639
[MindSPONGE] Step: 210, E_pot: 10043.269
[MindSPONGE] Step: 220, E_pot: 9262.627
[MindSPONGE] Step: 23

In [9]:
vgen = VelocityGenerator(300)
velocity = vgen(system.shape, system.atom_mass)

In [10]:
opt = UpdaterMD(system=system,
                time_step=1e-3,
                velocity=velocity,
                integrator='velocity_verlet',
                temperature=300,
                thermostat='langevin',)

In [11]:
md = Sponge(system, energy, min_opt)

[MindSPONGE] Using fully connected neighbour list (not updated).


In [12]:
cb_h5md = WriteH5MD(system, 'tutorial_b06.h5md', save_freq=10, write_velocity=True, write_force=True)

In [13]:
md.change_optimizer(opt)
md.run(2000, callbacks=[run_info, cb_h5md])

[MindSPONGE] Started simulation at 2023-06-30 16:50:02
[MindSPONGE] Step: 0, E_pot: -1609.2458, E_kin: 9305.716, E_tot: 7696.4697, Temperature: 295.15308
[MindSPONGE] Step: 10, E_pot: -2079.5398, E_kin: 9494.258, E_tot: 7414.718, Temperature: 301.13312
[MindSPONGE] Step: 20, E_pot: -3061.2454, E_kin: 11006.04, E_tot: 7944.795, Temperature: 349.08292
[MindSPONGE] Step: 30, E_pot: -1711.0934, E_kin: 9731.676, E_tot: 8020.5825, Temperature: 308.6634
[MindSPONGE] Step: 40, E_pot: -3201.4827, E_kin: 11231.6455, E_tot: 8030.163, Temperature: 356.23853
[MindSPONGE] Step: 50, E_pot: -2869.9229, E_kin: 10555.592, E_tot: 7685.669, Temperature: 334.79587
[MindSPONGE] Step: 60, E_pot: -3077.96, E_kin: 10582.215, E_tot: 7504.255, Temperature: 335.6403
[MindSPONGE] Step: 70, E_pot: -3289.6162, E_kin: 10907.554, E_tot: 7617.9375, Temperature: 345.95917
[MindSPONGE] Step: 80, E_pot: -2895.5718, E_kin: 10423.396, E_tot: 7527.8247, Temperature: 330.60297
[MindSPONGE] Step: 90, E_pot: -3566.5032, E_kin: 